In [2]:
import pandas as pd
import numpy as np
import glob
import os

# inputs

In [34]:
input_idf_table_file_path = './Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv'
input_temp_pattern_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv'
input_storm_stats_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv'
output_dir = './Assignment_3__Output_FE__1st__centroid/'

print(
    input_idf_table_file_path,
    input_temp_pattern_file_path,
    input_storm_stats_file_path,
    sep='\n'
    )

./Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv


In [35]:
def create_output_dir(arg_output_dir):
    """create output directory if it does not exist

    arguments:
        [string] --> arg_output_dir = path of the output directory name
    """
    if not os.path.exists(arg_output_dir):
        os.makedirs(arg_output_dir)

In [36]:
create_output_dir(output_dir)
output_dir

'./Assignment_3__Output_FE__1st__centroid/'

# building idf table

In [37]:
idf_table = pd.read_csv(filepath_or_buffer=input_idf_table_file_path, skiprows=9)
frequency_label = idf_table.columns[2:].to_numpy()
idf_table.columns = idf_table.columns.to_series().apply(
    func=lambda arg: 
        '_'.join(arg.split(sep=' ')) 
        if arg[:8] == 'Duration' 
        else 'freq_' + '_'.join('_perc'.join('_'.join(arg.split(sep=' ')).split(sep='%')).split(sep='.'))
    )
frequency_tag = idf_table.columns[2:].to_numpy()
idf_table[['Duration', 'units']] = idf_table.apply(func=lambda arg: arg.Duration.split(sep=' '), axis=1, result_type='expand')
idf_table = idf_table[idf_table.columns[[0,-1] + list(range(1,len(idf_table.columns) - 1))]]
idf_table.Duration = idf_table.Duration.astype(float)
idf_table.to_csv(path_or_buf='{}table_idf_depths.csv'.format(output_dir), index=False)

idf_table

,Duration,units,Duration_in_min,freq_12EY,freq_6EY,freq_4EY,freq_3EY,freq_2EY,freq_63_2_perc,freq_50_perc,...,freq_20_perc,freq_0_2EY,freq_10_perc,freq_5_perc,freq_2_perc,freq_1_perc,freq_1_in_200,freq_1_in_500,freq_1_in_1000,freq_1_in_2000
0,1.0,min,1.0,0.999,1.13,1.37,1.55,1.80,2.28,2.51,...,3.25,3.31,3.74,4.22,4.84,5.32,5.82,6.58,7.17,7.76
1,2.0,min,2.0,1.770,2.00,2.39,2.67,3.07,3.77,4.10,...,5.14,5.25,5.86,6.57,7.52,8.25,9.08,10.30,11.30,12.20
2,3.0,min,3.0,2.410,2.73,3.29,3.68,4.25,5.23,5.71,...,7.21,7.35,8.23,9.24,10.60,11.60,12.80,14.50,15.80,17.10
3,4.0,min,4.0,2.960,3.36,4.07,4.58,5.30,6.56,7.19,...,9.15,9.33,10.50,11.80,13.50,14.90,16.30,18.40,20.10,21.80
4,5.0,min,5.0,3.430,3.91,4.76,5.37,6.23,7.75,8.52,...,10.90,11.10,12.60,14.10,16.20,17.80,19.50,22.10,24.00,26.10
5,10.0,min,10.0,5.150,5.91,7.28,8.27,9.70,12.20,13.60,...,17.60,18.00,20.40,23.00,26.40,29.00,31.60,35.80,39.00,42.20
6,15.0,min,15.0,6.290,7.25,8.96,10.20,12.00,15.30,16.90,...,22.00,22.50,25.50,28.70,33.00,36.20,39.60,44.70,48.70,52.80
7,20.0,min,20.0,7.160,8.26,10.20,11.70,13.70,17.50,19.40,...,25.20,25.70,29.10,32.80,37.70,41.40,45.20,51.20,55.70,60.40
8,25.0,min,25.0,7.860,9.07,11.20,12.80,15.10,19.30,21.40,...,27.70,28.30,31.90,36.00,41.30,45.30,49.60,56.10,61.20,66.30
9,30.0,min,30.0,8.460,9.76,12.10,13.80,16.30,20.80,23.00,...,29.70,30.30,34.20,38.60,44.30,48.60,53.20,60.20,65.60,71.10


# building temporal pattern table

In [63]:
temp_pattern_table = pd.read_csv(filepath_or_buffer=input_temp_pattern_file_path)
temp_pattern_table.rename(columns=lambda arg: arg.strip(), inplace=True)
temp_pattern_table.rename(columns={'Duration':'Duration_in_min'}, inplace=True)
temp_pattern_table.columns = pd.Index(data=temp_pattern_table.columns[:5].to_list() + ['Increment_{:02}'.format(ind) for ind in range(len(temp_pattern_table.columns[5:]))])
temp_pattern_table.to_csv(path_or_buf='{}table_temp_patterns.csv'.format(output_dir))

temp_pattern_table

,EventID,Duration_in_min,TimeStep,Region,AEP,Increment_00,Increment_01,Increment_02,Increment_03,Increment_04,...,Increment_46,Increment_47,Increment_48,Increment_49,Increment_50,Increment_51,Increment_52,Increment_53,Increment_54,Increment_55
0,4380,10,5,East Coast (South),frequent,58.06,41.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4382,10,5,East Coast (South),frequent,52.13,47.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4384,10,5,East Coast (South),frequent,60.71,39.29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4385,10,5,East Coast (South),frequent,51.51,48.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4386,10,5,East Coast (South),frequent,54.55,45.45,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856,10080,180,East Coast (South),rare,1.14,1.80,1.36,1.29,3.20,...,0.01,0.01,0.02,0.03,0.32,0.17,0.52,0.00,0.02,0.00
716,5064,10080,180,East Coast (South),rare,0.61,0.93,0.26,0.04,2.91,...,5.27,0.31,0.02,1.21,1.57,0.08,0.05,0.15,0.40,0.11
717,5069,10080,180,East Coast (South),rare,0.00,0.00,0.00,1.06,0.04,...,0.22,0.20,0.57,0.06,2.06,0.00,0.03,0.18,0.12,3.09
718,5071,10080,180,East Coast (South),rare,1.21,0.00,0.00,0.00,0.00,...,0.05,0.56,0.70,3.31,0.10,0.11,0.00,0.00,0.00,0.15


In [100]:
storm_stats_table = pd.read_csv(filepath_or_buffer=input_storm_stats_file_path)
storm_stats_table.rename(columns=lambda arg: arg.strip(), inplace=True)
for ind in [(' ','_'), ('(',''), (')',''), ('%','in_perc'), ('_No.','_Number'), ('_min','_in_min'), ('_mm','_in_mm')]:
    storm_stats_table.rename(columns=lambda arg: arg.replace(*ind), inplace=True)
storm_stats_table.rename(columns={'Burst_Duration_in_min':'Duration_in_min'}, inplace=True)
storm_stats_table.dropna(axis=0, inplace=True)
storm_stats_table.to_csv(path_or_buf='{}table_storm_stats.csv'.format(output_dir))
# storm_stats_table.Burst_Start_Date
# storm_stats_table.Burst_End_Date

storm_stats_table

,Event_ID,Region,Region_source,Duration_in_min,Burst_Loading,Original_Burst_Depth_in_mm,AEP_Window,AEP_source_in_perc,Burst_Start_Date,Burst_End_Date,DB_Event_Reference_Number,DB_Pluviograph_Reference_Number,Offical_Gauge,Lat,Long
0,4380.0,East Coast (South),East Coast (South),10.0,1.0,15.50,frequent,14.4679,3/25/2009 17:45,3/25/2009 17:50,113650.0,1828.0,qcd_563064_233,-32.2375,150.6306
1,4382.0,East Coast (South),East Coast (South),10.0,1.0,18.80,frequent,14.4924,12/28/1999 17:05,12/28/1999 17:10,39777.0,517.0,qcd_061250,-32.6296,151.5919
2,4384.0,East Coast (South),East Coast (South),10.0,1.0,16.80,frequent,14.5144,2/4/2005 9:30,2/4/2005 9:35,42925.0,556.0,qcd_068102,-34.4869,150.4019
3,4385.0,East Coast (South),East Coast (South),10.0,2.0,18.50,frequent,15.6473,3/27/1999 22:30,3/27/1999 22:35,114063.0,1835.0,qcd_563079_231,-33.7944,150.5083
4,4386.0,East Coast (South),East Coast (South),10.0,2.0,11.00,frequent,47.6798,4/28/1993 14:05,4/28/1993 14:10,113366.0,1824.0,qcd_563056_231,-34.0333,150.2153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856.0,East Coast (South),East Coast (North),10080.0,1.0,595.23,rare,1.0865,5/1/1996 0:05,5/8/1996 0:00,26738.0,351.0,qcd_040223,-27.4178,153.1142
716,5064.0,East Coast (South),East Coast (North),10080.0,2.0,589.88,rare,1.2899,1/6/1968 17:10,1/13/1968 17:05,24711.0,329.0,qcd_040093,-26.1831,152.6414
717,5069.0,East Coast (South),East Coast (North),10080.0,2.0,491.26,rare,2.7020,1/6/1968 4:10,1/13/1968 4:05,23354.0,313.0,qcd_039128,-24.9069,152.3230
718,5071.0,East Coast (South),East Coast (North),10080.0,2.0,400.84,rare,2.9538,1/7/1968 11:45,1/14/1968 11:40,27412.0,358.0,qcd_040318,-27.0258,152.5642


In [99]:
storm_stats_table.dtypes

Event_ID                           float64
Region                              object
Region_source                       object
Duration_in_min                    float64
Burst_Loading                      float64
Original_Burst_Depth_in_mm         float64
AEP_Window                          object
AEP_source_in_perc                 float64
Burst_Start_Date                    object
Burst_End_Date                      object
DB_Event_Reference_Number          float64
DB_Pluviograph_Reference_Number    float64
Offical_Gauge                       object
Lat                                float64
Long                               float64
dtype: object

In [7]:
print(frequency_label, frequency_tag, sep='\n')

['12EY' '6EY' '4EY' '3EY' '2EY' '63.2%' '50%' '0.5EY' '20%' '0.2EY' '10%'
 '5%' '2%' '1%' '1 in 200' '1 in 500' '1 in 1000' '1 in 2000']
['freq_12EY' 'freq_6EY' 'freq_4EY' 'freq_3EY' 'freq_2EY' 'freq_63_2_perc'
 'freq_50_perc' 'freq_0_5EY' 'freq_20_perc' 'freq_0_2EY' 'freq_10_perc'
 'freq_5_perc' 'freq_2_perc' 'freq_1_perc' 'freq_1_in_200' 'freq_1_in_500'
 'freq_1_in_1000' 'freq_1_in_2000']


# building frequency classification table

In [8]:
ey_constants = np.array(object=[12,6,4,3,2,1,0.5,0.2])
ey_constants

array([12. ,  6. ,  4. ,  3. ,  2. ,  1. ,  0.5,  0.2])

In [9]:
aep_variables = np.multiply(np.add(1, np.negative(np.exp(np.negative(ey_constants)))),100)
aep_variables

array([99.99938558, 99.75212478, 98.16843611, 95.02129316, 86.46647168,
       63.21205588, 39.34693403, 18.12692469])

In [10]:
aep_constants = np.array(object=[50, 20, 10, 5, 2, 1, 0.5, 0.2, 0.1, 0.05, 0.02])
aep_constants

array([5.e+01, 2.e+01, 1.e+01, 5.e+00, 2.e+00, 1.e+00, 5.e-01, 2.e-01,
       1.e-01, 5.e-02, 2.e-02])

In [11]:
ey_variables = np.negative(np.log(np.add(1, np.negative(np.divide(aep_constants, 100)))))
ey_variables

array([6.93147181e-01, 2.23143551e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [12]:
ey = np.flip(m=np.sort(a=np.concatenate((ey_constants, ey_variables))))
ey

array([1.20000000e+01, 6.00000000e+00, 4.00000000e+00, 3.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 6.93147181e-01, 5.00000000e-01,
       2.23143551e-01, 2.00000000e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [13]:
aep_percentage = np.flip(m=np.sort(a=np.concatenate((aep_constants, aep_variables))))
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [14]:
aep_1_in_x = np.divide(100, aep_percentage)
aep_1_in_x

array([1.00000614e+00, 1.00248491e+00, 1.01865736e+00, 1.05239570e+00,
       1.15651764e+00, 1.58197671e+00, 2.00000000e+00, 2.54149408e+00,
       5.00000000e+00, 5.51665557e+00, 1.00000000e+01, 2.00000000e+01,
       5.00000000e+01, 1.00000000e+02, 2.00000000e+02, 5.00000000e+02,
       1.00000000e+03, 2.00000000e+03, 5.00000000e+03])

In [15]:
ari = np.divide(1,ey)
ari

array([8.33333333e-02, 1.66666667e-01, 2.50000000e-01, 3.33333333e-01,
       5.00000000e-01, 1.00000000e+00, 1.44269504e+00, 2.00000000e+00,
       4.48142012e+00, 5.00000000e+00, 9.49122158e+00, 1.94957257e+01,
       4.94983165e+01, 9.94991625e+01, 1.99499582e+02, 4.99499833e+02,
       9.99499917e+02, 1.99949996e+03, 4.99949998e+03])

In [16]:
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [17]:
frequency_window = np.array(object=['frequent', 'Intermediate', 'Rare'])[
    np.add(np.digitize(x=aep_percentage, bins=np.array(object=[100, 14.4, 3.2, 0]), right=False), np.negative(1))
    ]
frequency_window

array(['frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'Intermediate', 'Intermediate', 'Rare', 'Rare', 'Rare', 'Rare',
       'Rare', 'Rare', 'Rare'], dtype='<U12')